בכדי להריץ את הקוד יש להוריד את קבצי הדאטה שצירפתי בהגשה לגוגל דרייב

Mounts Google Drive and sets up tools to display videos in Colab.

In [ ]:
from google.colab import drive

# Mount Google Drive to access dataset and save/load models
drive.mount('/content/drive')

Imports tools for building and evaluating an LSTM classification model.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report
from tensorflow.keras.callbacks import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import random
from tensorflow.keras.models import load_model
tf.config.run_functions_eagerly(True)

set paths and labels

In [ ]:
annotated_train_path = "/content/drive/MyDrive/ASL_Project_Mika/Dataset_submission/annotated_train_data"
augmented_train_path = "/content/drive/MyDrive/ASL_Project_Mika/Dataset_submission/augmented_train_data"
test_path = "/content/drive/MyDrive/ASL_Project_Mika/Dataset_submission/annotated_test_data"
allowed_labels = {'book', 'drink', 'computer', 'study','science','i', 'water', 'read', 'other'}  # Only use these classes

Set fixed values

In [ ]:
MAX_FRAMES = 30  # Fixed sequence length
FEATURE_SIZE = 150  # Fixed feature size
LEARNING_RATE = 0.001  # Custom learning rate

# Argumantion testing

Function to load sequences and labels

In [ ]:
def load_sequences_from_multiple_paths(folders, max_per_class=150):
    sequences = []  # List to store all sequences (each is a (30, FEATURE_SIZE) matrix)
    labels = []     # List to store corresponding class labels
    label_encoder = None  # Will be used to encode class names as numeric labels

    # Collect all class names from allowed folders
    class_names_set = set()
    for folder in folders:
        # Only include subfolder names that are in allowed_labels
        class_names_set.update(name for name in os.listdir(folder) if name in allowed_labels)

    # Sort class names for consistent label encoding
    class_names = sorted(list(class_names_set))
    print(f"Class names: {class_names}")

    # Initialize and fit the label encoder on the collected class names
    label_encoder = LabelEncoder()
    label_encoder.fit(class_names)

    # Iterate over each folder (e.g., train and test folders)
    for folder in folders:
        # For each valid class name (folder)
        for class_name in class_names:
            class_path = os.path.join(folder, class_name) # Full path to class folder
            if os.path.isdir(class_path): # Proceed only if it's a directory
                files = [f for f in os.listdir(class_path) if f.endswith(".csv")]
                files = sorted(files)[:max_per_class]  # Limit per class per folder

                for filename in files:
                    file_path = os.path.join(class_path, filename)
                    try:
                        # Load CSV file as DataFrame
                        df = pd.read_csv(file_path, header=None)

                        # Ensure feature size matches expected
                        if df.shape[1] < FEATURE_SIZE:
                            # Pad columns with zeros if there are fewer features
                            padding = np.zeros((df.shape[0], FEATURE_SIZE - df.shape[1]))
                            df = pd.concat([df, pd.DataFrame(padding)], axis=1)
                        elif df.shape[1] > FEATURE_SIZE:
                            # Trim excess features
                            df = df.iloc[:, :FEATURE_SIZE]

                        # Select up to MAX_FRAMES from the sequence, evenly spaced
                        total_frames = df.shape[0]
                        step = max(1, total_frames // MAX_FRAMES)
                        selected_frames = df.iloc[::step].values[:MAX_FRAMES]

                        # If there are not enough frames, pad with zeros
                        if selected_frames.shape[0] < MAX_FRAMES:
                            padding = np.zeros((MAX_FRAMES - selected_frames.shape[0], FEATURE_SIZE))
                            selected_frames = np.vstack([selected_frames, padding])

                        # Save processed sequence and corresponding label
                        sequences.append(selected_frames)
                        labels.append(label_encoder.transform([class_name])[0])

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                        continue

    # Convert final sequence and label lists to NumPy arrays
    X = np.array(sequences)  # Shape: (num_samples, MAX_FRAMES, FEATURE_SIZE)
    y = np.array(labels)     # Shape: (num_samples,)

    print(f" Loaded combined dataset: X={X.shape}, y={y.shape}")
    return X, y, label_encoder

Load train and test datasets

In [ ]:
X_train, y_train, label_encoder = load_sequences_from_multiple_paths([annotated_train_path, augmented_train_path], max_per_class=150)
X_test, y_test, _ = load_sequences_from_multiple_paths([test_path], max_per_class=30)

# Check if dataset is loaded properly
if X_train.shape[0] == 0 or X_test.shape[0] == 0:
    raise ValueError("No valid training or testing data found. Please check dataset format.")

Define an LSTM-based sequence classification model

In [ ]:
model = Sequential([
    # Mask padded frames
    Masking(mask_value=0.0, input_shape=(X_train.shape[1], X_train.shape[2])),

    # First LSTM layer (return sequences for stacking)
    LSTM(128, return_sequences=True),
    Dropout(0.3),                         # Dropout after first LSTM
    BatchNormalization(),                # Batch norm helps stabilize deeper LSTM

    # Second LSTM layer (final sequence output)
    LSTM(64),
    Dropout(0.3),

    # Dense hidden layer
    Dense(64, activation='relu'),
    Dropout(0.3),                         # Optional dropout before final output

    # Output layer
    Dense(len(label_encoder.classes_), activation='softmax')
])

Set custom learning rate and compile the model with the custom learning rat

In [ ]:
optimizer = Adam(learning_rate=LEARNING_RATE)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Track the best epoch for confusion matrix

In [ ]:
best_epoch = 0
best_val_accuracy = 0.0
best_y_pred = None

Custom callback to track precision, recall, and store the best model
predictions based on validation accuracy



In [ ]:
class PrecisionRecallCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Use global variables to keep track of the best performing epoch
        global best_epoch, best_val_accuracy, best_y_pred

        # Get model predictions for training and test sets
        y_pred_train = np.argmax(self.model.predict(X_train), axis=1)
        y_pred_test = np.argmax(self.model.predict(X_test), axis=1)

        # Compute precision and recall for training data
        train_precision = precision_score(y_train, y_pred_train, average='weighted', zero_division=0)
        train_recall = recall_score(y_train, y_pred_train, average='weighted', zero_division=0)

        # Compute precision and recall for test (validation) data
        test_precision = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
        test_recall = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

        # Get current epoch's validation accuracy (automatically logged by Keras)
        val_accuracy = logs["val_accuracy"]

        # Print summary of precision and recall for this epoch
        print(f"\n Epoch {epoch+1}: Train Precision={train_precision:.4f}, Train Recall={train_recall:.4f}, "
              f"Test Precision={test_precision:.4f}, Test Recall={test_recall:.4f}")

        # Track best epoch
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy   # Update the best accuracy
            best_epoch = epoch + 1             # Save the best epoch (1-indexed)
            best_y_pred = y_pred_test.copy()   # Save the best predictions

Creates a callback to save the model to Google Drive whenever validation accuracy improves during training.

In [ ]:
checkpoint_callback = ModelCheckpoint(
    "/content/drive/MyDrive/ASL_Project_Mika/Model/ASL_final_model.h5",
    monitor='val_accuracy',  # Track validation accuracy during training
    save_best_only=True,     # Only save the model if val_accuracy improves
    save_weights_only=False, # Save the entire model (not just weights)
    verbose=1                # Print a message each time the model is saved
)

Train and save model

In [ ]:
# Train model and save history
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=40,
    batch_size=16,
    callbacks=[PrecisionRecallCallback(), checkpoint_callback]
)

Evaluate best epoch with confusion matrix

In [ ]:
print(f"\n Best Epoch: {best_epoch}, Best Validation Accuracy: {best_val_accuracy:.4f}")

Compute confusion matrix

In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, best_y_pred)

# Plot with adjustments
plt.figure(figsize=(10, 8))  # Larger figure for spacing
sns.heatmap(conf_matrix,
            annot=True,
            fmt='d',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cmap='Blues')

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")

# Rotate the tick labels for clarity
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()  # Prevent clipping
plt.show()

Classification report

In [ ]:
report = classification_report(y_test, best_y_pred, target_names=label_encoder.classes_, digits=4)

print("\n🔹 Classification Report:\n", report)

Plots the training and validation accuracy over epochs to visualize the model’s learning performance.

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy', color='lightpink')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', color='lightblue')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

Plots the training and validation loss over epochs to track how well the model is minimizing error.

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss', color='lightpink')
plt.plot(history.history['val_loss'], label='Val Loss', color='lightblue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

Loads the trained model, predicts on the test set, calculates precision and recall per class, and visualizes them in a bar chart.

In [ ]:
# --- Load model (adjust path if needed) ---
model = load_model('/content/drive/MyDrive/ASL_Project_Mika/Model/ASL_final_model.h5')  # or .h5

# --- Predict on test set ---
y_pred = np.argmax(model.predict(X_test), axis=1)

# --- Calculate precision and recall per class ---
precision = precision_score(y_test, y_pred, average=None, zero_division=0)
recall = recall_score(y_test, y_pred, average=None, zero_division=0)

# --- Get class labels ---
class_names = label_encoder.classes_

# --- Plot ---
x = np.arange(len(class_names))
width = 0.35  # Bar width

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, precision, width, label='Precision', color='#ffb6c1')  # Light pink
plt.bar(x + width/2, recall, width, label='Recall', color='#add8e6')        # Light blue

plt.xlabel('ASL Word')
plt.ylabel('Score')
plt.title('Precision and Recall per ASL Class')
plt.xticks(x, class_names, rotation=45)
plt.ylim(0, 1)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()